# Convolutional Neural Networks

This note book will be used to test convolutional neural networks on each of the 3 ontologies.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import itertools
import pprint
import yaml

2024-02-23 18:24:14.076768: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 18:24:14.076873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 18:24:14.156840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 18:24:14.187131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 18:24:24.089634: W tensorflow/compiler/tf2

In [2]:
print("TensorFlow v" + tf.__version__)
print("Numpy v" + np.__version__)

TensorFlow v2.15.0
Numpy v1.26.4


In [18]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
config_path = './config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [4]:
USE_75_PERCENT_DATA = config['use_75_percent_datasets']
partial_dataset_prefix = '75percent_' if USE_75_PERCENT_DATA else ''
BP_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_BiologicalProcesses.pkl")
CC_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_CellularComponent.pkl")
MF_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_MolecularFunction.pkl")
BP_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_BiologicalProcesses.pkl")
CC_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_CellularComponent.pkl")
MF_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_MolecularFunction.pkl")

In [5]:
train_data_dict = {'Biological Processes': [BP_train_df, BP_label_df],
                   'Cellular Component': [CC_train_df, CC_label_df],
                   'Molecular Function': [MF_train_df, MF_label_df]
}

In [6]:
num_labels = 1500
num_folds = config['num_folds']

## Model 1 Architecture: CNN

In [7]:
BATCH_SIZE = config['batch_size']

In [8]:
def model1_training(dataset_name, data):
  train = data[0]
  label = data[1]

  INPUT_SHAPE = (train.shape[1], 1)

  best_f1 = 0
  print('=======================================================================')
  print(f'Training for {dataset_name}')
  
  model_root_path = f'{config["directories"]["models"]}/CNNMod1'

  for num_filters, size_kernel in itertools.product([16, 32], [64, 128, 256]):
    print('----------------------------------------------------------------------')
    print('The number of filters is ', num_filters)
    print('The size of the kernel is ', size_kernel)

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1

    for train_fold, test_fold in kfold.split(train, label):

      model1 = tf.keras.Sequential([
          tf.keras.layers.BatchNormalization(input_shape = INPUT_SHAPE),
          tf.keras.layers.Conv1D(filters = num_filters, kernel_size = size_kernel, activation = 'relu'),
          tf.keras.layers.MaxPooling1D(pool_size = 2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(units = 512, activation = 'relu'),
          tf.keras.layers.Dense(units = num_labels, activation = 'sigmoid')
          ])

      # Compile model
      model1.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
          loss='binary_crossentropy',
          metrics=['binary_accuracy',
                  tf.keras.metrics.AUC(),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall(),
                  ] # tf.keras.metrics.F1Score() not appropriate as it is calculated batchwise
          )

      print(f'Training for fold {fold_no} ...')

      # Fit the data to the model
      history = model1.fit(
          train.iloc[train_fold], label.iloc[train_fold],
          validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
          batch_size=BATCH_SIZE,
          epochs=5
          )

      # Generate metrics
      scores = model1.evaluate(train.iloc[test_fold], label.iloc[test_fold], verbose=0)
      precision = scores[3]
      recall = scores[4]
      F1_score = 2*precision*recall / (precision + recall)
      print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model1.metrics_names[0]} of {scores[0]}; {model1.metrics_names[1]} of {scores[1]*100}%')

      if F1_score > best_f1:
        best_f1 = F1_score
        if dataset_name == 'Biological Processes':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_BP_model')
          print(f'Current best model for Biological Processes has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

        elif dataset_name == 'Molecular Function':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_MF_model')
          print(f'Current best model for Molecular Function has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

        else:
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_CC_model')
          print(f'Current best model for Cellular Component has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

      fold_no += 1

In [9]:
for dataset in train_data_dict:
  dataset_name = dataset
  data = train_data_dict[dataset]
  model1_training(dataset_name, data)

Training for Biological Processes
----------------------------------------------------------------------
The number of filters is  16
The size of the kernel is  64


2024-02-23 18:24:52.798707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79086 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:44:00.0, compute capability: 8.0


Training for fold 1 ...
Epoch 1/5


2024-02-23 18:24:58.443054: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-23 18:25:05.297466: I external/local_xla/xla/service/service.cc:168] XLA service 0x2b487b9c0ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-23 18:25:05.297534: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-02-23 18:25:05.309087: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708705505.923059  113418 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


147/147 [==============================] - 19s 15ms/step - loss: 0.1126 - binary_accuracy: 0.9669 - auc: 0.8153 - precision: 0.3018 - recall: 0.1221 - val_loss: 0.4684 - val_binary_accuracy: 0.9736 - val_auc: 0.8738 - val_precision: 0.6710 - val_recall: 0.1480
Epoch 2/5
147/147 [==============================] - 1s 9ms/step - loss: 0.0866 - binary_accuracy: 0.9742 - auc: 0.8942 - precision: 0.6723 - recall: 0.1860 - val_loss: 0.2463 - val_binary_accuracy: 0.9741 - val_auc: 0.9083 - val_precision: 0.6693 - val_recall: 0.1871
Epoch 3/5
147/147 [==============================] - 1s 10ms/step - loss: 0.0816 - binary_accuracy: 0.9746 - auc: 0.9138 - precision: 0.6716 - recall: 0.2172 - val_loss: 0.1611 - val_binary_accuracy: 0.9740 - val_auc: 0.9148 - val_precision: 0.6125 - val_recall: 0.2445
Epoch 4/5
147/147 [==============================] - 1s 9ms/step - loss: 0.0791 - binary_accuracy: 0.9749 - auc: 0.9217 - precision: 0.6738 - recall: 0.2338 - val_loss: 0.0923 - val_binary_accuracy: 0

...

Current best model for Molecular Function has 16 filters, with a kernel size of 64 and has an F1 score of 0.43271499873681657
Training for fold 2 ...
Epoch 1/5
98/98 [==============================] - 3s 13ms/step - loss: 0.1240 - binary_accuracy: 0.9630 - auc_121: 0.8070 - precision_121: 0.2980 - recall_121: 0.1495 - val_loss: 0.5399 - val_binary_accuracy: 0.9731 - val_auc_121: 0.8767 - val_precision_121: 0.7101 - val_recall_121: 0.1892
Epoch 2/5
98/98 [==============================] - 1s 8ms/step - loss: 0.0906 - binary_accuracy: 0.9732 - auc_121: 0.8949 - precision_121: 0.7140 - recall_121: 0.2135 - val_loss: 0.4088 - val_binary_accuracy: 0.9740 - val_auc_121: 0.9121 - val_precision_121: 0.7065 - val_recall_121: 0.2412
Epoch 3/5
98/98 [==============================] - 1s 9ms/step - loss: 0.0843 - binary_accuracy: 0.9739 - auc_121: 0.9162 - precision_121: 0.7148 - recall_121: 0.2509 - val_loss: 0.2824 - val_binary_accuracy: 0.9745 - val_auc_121: 0.9249 - val_precision_121: 0.7215 -

INFO:tensorflow:Assets written to: ./models/CNNMod1/best_MF_model/assets


Current best model for Molecular Function has 32 filters, with a kernel size of 64 and has an F1 score of 0.46599281869895987
Training for fold 4 ...
Epoch 1/5
98/98 [==============================] - 3s 14ms/step - loss: 0.1215 - binary_accuracy: 0.9626 - auc_153: 0.8142 - precision_153: 0.2938 - recall_153: 0.1573 - val_loss: 0.5436 - val_binary_accuracy: 0.9727 - val_auc_153: 0.8755 - val_precision_153: 0.7255 - val_recall_153: 0.1879
Epoch 2/5
98/98 [==============================] - 1s 11ms/step - loss: 0.0890 - binary_accuracy: 0.9735 - auc_153: 0.8985 - precision_153: 0.7132 - recall_153: 0.2256 - val_loss: 0.3984 - val_binary_accuracy: 0.9735 - val_auc_153: 0.9129 - val_precision_153: 0.7196 - val_recall_153: 0.2365
Epoch 3/5
98/98 [==============================] - 1s 10ms/step - loss: 0.0829 - binary_accuracy: 0.9742 - auc_153: 0.9188 - precision_153: 0.7145 - recall_153: 0.2642 - val_loss: 0.2830 - val_binary_accuracy: 0.9740 - val_auc_153: 0.9244 - val_precision_153: 0.7214

INFO:tensorflow:Assets written to: ./models/CNNMod1/best_MF_model/assets


Current best model for Molecular Function has 32 filters, with a kernel size of 256 and has an F1 score of 0.46617622430431505
Training for fold 3 ...
Epoch 1/5
98/98 [==============================] - 3s 18ms/step - loss: 0.1234 - binary_accuracy: 0.9622 - auc_172: 0.8100 - precision_172: 0.2813 - recall_172: 0.1496 - val_loss: 0.5001 - val_binary_accuracy: 0.9727 - val_auc_172: 0.8771 - val_precision_172: 0.7373 - val_recall_172: 0.1689
Epoch 2/5
98/98 [==============================] - 1s 13ms/step - loss: 0.0895 - binary_accuracy: 0.9734 - auc_172: 0.8976 - precision_172: 0.7144 - recall_172: 0.2217 - val_loss: 0.3792 - val_binary_accuracy: 0.9737 - val_auc_172: 0.9137 - val_precision_172: 0.7372 - val_recall_172: 0.2198
Epoch 3/5
98/98 [==============================] - 1s 12ms/step - loss: 0.0831 - binary_accuracy: 0.9741 - auc_172: 0.9188 - precision_172: 0.7128 - recall_172: 0.2631 - val_loss: 0.2678 - val_binary_accuracy: 0.9742 - val_auc_172: 0.9254 - val_precision_172: 0.710

## Model 2 Architecture: CNN with LSTM